In [81]:
import pymysql
import requests

下載分析pm2.5 opendata資料

In [82]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [83]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\Django_Lennox\MYSQL\pm25_tomysql\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [84]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '5',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '10',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '17',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '2',
  'datacreationdate'

In [21]:
len(datas)

1000

In [22]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '4',
 'datacreationdate': '2025-08-20 19:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [68]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""

print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [94]:
conn=pymysql.connect(
host="localhost",
user="root",
password="",
port=3307,
database="demo"
)

cursor=conn.cursor()
cursor

In [89]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [60]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '4', '2025-08-20 19:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',4,'2025-08-20 19:00','μg/m3')"

In [90]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '%s,%s,%s,%s,%s)' at line 1")

### 一次寫入
- 加上 ignore

In [91]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

#移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [87]:
conn.close()

In [92]:
import pandas as pd 

In [95]:
cursor.execute("select * from pm25")

datas=cursor.fetchall()
datas

((1, '員林', '彰化縣', 4, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (2, '大城', '彰化縣', 5, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (3, '富貴角', '新北市', 5, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4, '麥寮', '雲林縣', 1, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (5, '關山', '臺東縣', 4, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (6, '馬公', '澎湖縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (7, '馬祖', '連江縣', 13, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (8, '埔里', '南投縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (9, '復興', '高雄市', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (10, '永和', '新北市', 8, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (11, '竹山', '南投縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (12, '中壢', '桃園市', 10, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (13, '三重', '新北市', 11, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (14, '冬山', '宜蘭縣', 9, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (15, '宜蘭',

In [103]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])

df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,2025-08-20 19:00:00
1,大城,2025-08-20 19:00:00
2,富貴角,2025-08-20 19:00:00
3,麥寮,2025-08-20 19:00:00
4,關山,2025-08-20 19:00:00
...,...,...
546,新莊,2025-08-20 20:00:00
547,板橋,2025-08-20 20:00:00
548,土城,2025-08-20 20:00:00
549,汐止,2025-08-20 20:00:00
